##### Imports

In [2]:
import cv2 as cv
import tools
import random
from os.path import join
import degpackage
import enhpackage
import qualitypackage
import edgepackage

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

##### Data Loader

In [2]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

rgb_dir = 'dataset/rgb'
boundary_dir = 'dataset/boundary'
batch_size = 9229

rgb_names, boundary_names = tools.im_exists(rgb_dir, batch_size)

dataset_rgb = []
dataset_depth = []
square_distances = []
i_ys = []

for image in rgb_names:
  dataset_rgb.append(cv.imread(join('dataset/rgb', image)))
  dataset_depth.append(cv.imread(join('dataset/depth', image)))

for image in boundary_names:
  boundary = cv.imread(join('dataset/boundary', image))[:, :, 0].astype(np.uint8)
  distance = cv.distanceTransform(cv.bitwise_not(boundary), cv.DIST_L1, maskSize=3)
  square_distances.append(distance * distance)
  i_ys.append(np.sum(boundary) / 255)


##### Degradation and Enhancement

In [3]:
deg_paths = ['deg_haze', 'deg_haze_depth', 'deg_random_noise', 'deg_reduce_contr_bright', 'deg_none']
# enh_paths = ['enh_none', 'enh_increase_contr_bright']
enh_paths = ['enh_clahe', 'enh_hist_eq']
img_paths = []

# for method_deg in deg_paths:
#   degpackage.degrade(method_deg, dataset_rgb, dataset_depth, rgb_names)

for method_enh in enh_paths:
  for method_deg in deg_paths:
    enhpackage.enhance(method_enh, method_deg, rgb_names)


##### Quality Evaluation

In [4]:
for deg in deg_paths:
  for enh in enh_paths:
    img_paths.append(join(deg, enh))

for path in img_paths:
  qualitypackage.eval_quality(dataset_rgb, path, rgb_names)

Data successfully writen to:  deg_haze\enh_clahe\_quality.csv
Data successfully writen to:  deg_haze\enh_hist_eq\_quality.csv
Data successfully writen to:  deg_haze_depth\enh_clahe\_quality.csv
Data successfully writen to:  deg_haze_depth\enh_hist_eq\_quality.csv
Data successfully writen to:  deg_random_noise\enh_clahe\_quality.csv
Data successfully writen to:  deg_random_noise\enh_hist_eq\_quality.csv
Data successfully writen to:  deg_reduce_contr_bright\enh_clahe\_quality.csv
Data successfully writen to:  deg_reduce_contr_bright\enh_hist_eq\_quality.csv
Data successfully writen to:  deg_none\enh_clahe\_quality.csv
Data successfully writen to:  deg_none\enh_hist_eq\_quality.csv


##### Edge Detection

In [5]:
for path in img_paths:

  modded_rgb = []

  for rgb in rgb_names:
    modded_rgb.append(cv.imread(join(path, rgb)))

  modded_boundary = edgepackage.edge_detection(path, modded_rgb, boundary_names)
  edgepackage.edge_evaluation(path, modded_boundary, boundary_names, square_distances, i_ys)

Images writen to:  deg_haze\enh_clahe
Data successfully writen to:  deg_haze\enh_clahe\_edge.csv
Images writen to:  deg_haze\enh_hist_eq
Data successfully writen to:  deg_haze\enh_hist_eq\_edge.csv
Images writen to:  deg_haze_depth\enh_clahe
Data successfully writen to:  deg_haze_depth\enh_clahe\_edge.csv
Images writen to:  deg_haze_depth\enh_hist_eq
Data successfully writen to:  deg_haze_depth\enh_hist_eq\_edge.csv
Images writen to:  deg_random_noise\enh_clahe
Data successfully writen to:  deg_random_noise\enh_clahe\_edge.csv
Images writen to:  deg_random_noise\enh_hist_eq
Data successfully writen to:  deg_random_noise\enh_hist_eq\_edge.csv
Images writen to:  deg_reduce_contr_bright\enh_clahe
Data successfully writen to:  deg_reduce_contr_bright\enh_clahe\_edge.csv
Images writen to:  deg_reduce_contr_bright\enh_hist_eq
Data successfully writen to:  deg_reduce_contr_bright\enh_hist_eq\_edge.csv
Images writen to:  deg_none\enh_clahe
Data successfully writen to:  deg_none\enh_clahe\_edge

##### Delete

In [11]:
img_gray = cv.imread('dataset/rgb/09831.png', cv.IMREAD_GRAYSCALE)
sobx = cv.Sobel(img_gray, cv.CV_64F, 1, 0, ksize=5)
soby = cv.Sobel(img_gray, cv.CV_64F, 0, 1, ksize=5)
sob = sobx+soby
cv.imwrite('report_images/Sobel_09831.png', sob)

plt.imshow(sobx)
plt.show()
plt.imshow(soby)
plt.show()
plt.imshow(sob)
plt.show()

error: OpenCV(4.9.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:94: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<1,-1,-1>,struct cv::impl::A0x59191d0d::Set<3,4,-1>,struct cv::impl::A0x59191d0d::Set<0,2,5>,4>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Unsupported depth of input image:
>     'VDepth::contains(depth)'
> where
>     'depth' is 6 (CV_64F)
